<a href="https://colab.research.google.com/github/JaMa-95/ML-Final/blob/master/ML_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/JaMa-95/ML-Final.git

Cloning into 'ML-Final'...
remote: Enumerating objects: 2, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 3063 (delta 0), reused 2 (delta 0), pack-reused 3061
Receiving objects: 100% (3063/3063), 187.64 MiB | 42.40 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [44]:
import numpy as np
import pandas as pd
import os
import cv2
import random
from tqdm import tqdm

from tensorflow.keras import applications
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

In [35]:
categories = ['dog', 'panda', 'cat']
X_train, X_test = [], []
y_train, y_test = [], []
imagePaths = []
HEIGHT = 32
WIDTH = 55
N_CHANNELS = 3

########################################################################
########Changed the file path because of google colab###################
########################################################################

# load training data
for k, category in enumerate(categories):
    for f in os.listdir('/content/ML-Final/data/train/' + category):
        imagePaths.append(['/content/ML-Final/data/train/' + category+'/'+f, k])

# loop over the input images
for imagePath in tqdm(imagePaths):
    if 'ds_store' in imagePath[0].lower():
        continue
    # load the image, resize the image to be HEIGHT * WIDTH pixels (ignoring
    # aspect ratio) and store the image in the data list
    image = cv2.imread(imagePath[0])
    image = cv2.resize(image, (WIDTH, HEIGHT))  # .flatten()
    X_train.append(image)
    # extract the class label from the image path and update the
    # labels list
    label = imagePath[1]
    y_train.append(label)




# load test data
imagePaths = []
for k, category in enumerate(categories):
    for f in os.listdir('/content/ML-Final/data/test/' + category):
        imagePaths.append(['/content/ML-Final/data/test/' + category+'/'+f, k])

# loop over the input images
for imagePath in tqdm(imagePaths):
    if 'ds_store' in imagePath[0].lower():
        continue
    # load the image, resize the image to be HEIGHT * WIDTH pixels (ignoring
    # aspect ratio) and store the image in the data list
    image = cv2.imread(imagePath[0])
    image = cv2.resize(image, (WIDTH, HEIGHT))  # .flatten()
    X_test.append(image)
    # extract the class label from the image path and update the
    # labels list
    label = imagePath[1]
    y_test.append(label)


X_train, X_test, y_train, y_test  = np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

############
# Create here your code. X_train, X_test, and so on is already given and splitted.

100%|██████████| 153/153 [00:00<00:00, 314.46it/s]


In [50]:
#DataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False, 
    samplewise_center=False,
    featurewise_std_normalization=False, 
    samplewise_std_normalization=False,
    zca_whitening=False, 
    zca_epsilon=1e-06, 
    rotation_range=30, 
    width_shift_range=0.1,
    height_shift_range=0.1, 
    brightness_range=None, 
    shear_range=0.0, 
    zoom_range=0.2,
    channel_shift_range=0.0, 
    fill_mode='nearest', 
    cval=0.0,
    horizontal_flip=True, 
    vertical_flip=False, 
    rescale=1./255,
    preprocessing_function=None, 
    data_format=None, 
    validation_split=0.0, 
    dtype=None
)

datagen_test = ImageDataGenerator(rescale=1./255, samplewise_center=True)
X_train.shape

(2879, 32, 55, 3)

In [ ]:
# Define our CNN
model = Sequential()
model.add(Conv2D(32, (3,3), strides=1, padding='same', activation='relu', input_shape=X_train.shape[1:])) # this layer has 32 filters. The filter size is: (3,3)
model.add(BatchNormalization()) # This is optional to avoid overfitting
model.add(MaxPool2D((2,2), strides=2, padding='same'))

model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.1)) # Dropout is optional.
model.add(BatchNormalization()) # BatchNormalization is optional
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='softmax')) # we use softmax with 5 output neurons, since we have 5 classes.

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # make sure we have categorical_crossentropy as loss
model.summary()


In [48]:
# fit the model, as known from last lecture.
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
90/90 [==============================] - 18s 187ms/step - loss: 0.0000e+00 - accuracy: 0.3168 - val_loss: 0.0000e+00 - val_accuracy: 0.3333
Epoch 2/10
90/90 [==============================] - 17s 184ms/step - loss: 0.0000e+00 - accuracy: 0.3272 - val_loss: 0.0000e+00 - val_accuracy: 0.3333
Epoch 3/10
90/90 [==============================] - 18s 195ms/step - loss: 0.0000e+00 - accuracy: 0.3309 - val_loss: 0.0000e+00 - val_accuracy: 0.3333
Epoch 4/10
90/90 [==============================] - 20s 216ms/step - loss: 0.0000e+00 - accuracy: 0.3303 - val_loss: 0.0000e+00 - val_accuracy: 0.3333
Epoch 5/10
16/90 [====>.........................] - ETA: 13s - loss: 0.0000e+00 - accuracy: 0.2826

KeyboardInterrupt: ignored

In [54]:
# load existing CNN from tensorflow. Here we use VGG16, pretrained on imagenet. we will not include_top.
# include_top means the flatten layer + the following dense layers. Wen only use the CNN-blocks from this network.
model = applications.VGG16(weights='imagenet', include_top=False, input_shape=X_train.shape[1:])

flat1 = Flatten()(model.output) # add a flatten to the VGG16
class1 = Dense(256, activation='relu')(flat1) # add a Dense layer after the flatten
output = Dense(1, activation='softmax')(class1) # add a dense layer after the 256-dense layer

model = Model(inputs=model.inputs, outputs=output) # define our final model. The first part is from VGG16 and the output is the layer defined above

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 32, 55, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 55, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 55, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 27, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 27, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 27, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 13, 128)        0  

In [55]:
model.fit(X_train, y_train, batch_size=32, epochs=3, validation_data=(X_test, y_test))

Epoch 1/3
90/90 [==============================] - 366s 4s/step - loss: 0.0000e+00 - accuracy: 0.3386 - val_loss: 0.0000e+00 - val_accuracy: 0.3333
Epoch 2/3
90/90 [==============================] - 361s 4s/step - loss: 0.0000e+00 - accuracy: 0.3411 - val_loss: 0.0000e+00 - val_accuracy: 0.3333
Epoch 3/3
65/90 [====================>.........] - ETA: 1:40 - loss: 0.0000e+00 - accuracy: 0.3208

KeyboardInterrupt: ignored